In [3]:
import openrouteservice as ors
import folium
from pandas import DataFrame
import pandas as pd
import math
import time

In [24]:
client = ors.Client(key='5b3ce3597851110001cf6248e87222039de944878943e770cf957d7d')

In [9]:
cur_dir = '../data/curated/'
raw_dir = '../data/raw/'

In [10]:

rent = pd.read_csv("../rent_data/renew(15)")
school = pd.read_csv(f"{cur_dir}school_df_21_preprocessed.csv")
mall =  pd.read_csv(f'{raw_dir}mall_info.csv')

In [11]:
def mapping(coordinates, client):
    map = folium.Map(location=[37.50, 144.58], tiles='cartodbpositron', zoom_start=13)
    route = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson',
        validate=False,
        )
    folium.PolyLine(locations=[list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']]).add_to(map)
    return route, map

In [100]:
def cal_distance(coordinates, client):
    matrix = client.distance_matrix(
    locations=coordinates,
    profile='driving-car',
    metrics=['distance', 'duration'],
    validate=False,
    )
    time.sleep(2)
    return matrix['durations'], matrix['distances']
# the unit of distance is m, the unit of durations is seconds



In [99]:
coordinates = []
for j in range(10):
        coordinates.append([school["X"][j], school["Y"][j]])

a = cal_distance(coordinates, client)[2]
a

TypeError: unhashable type: 'list'

In [13]:
def times(len):
    times_i = int(math.sqrt(3500))-1
    times = math.ceil(len/times_i)
    times_function = math.ceil((len+times)/times_i)
    return times_i, times_function

In [108]:
def get_coordinate(rent, school):
    rent_address = []
    school_name = []
    min_distance = []
    min_duration = []
    for i in range(len(rent)):
        coordinates = []
        distances = []
        durations = []
        for j in range(10):
            coordinates.append([school["X"][j], school["Y"][j]])
        print(coordinates)
        for k in range(times(len(school))[1]):
            coor_part = []
            coor_part.append([rent["longitude"][i], rent["latitude"][i]])
            coor_part.extend(coordinates[k*times(len(coordinates))[0]:(k+1)*times(len(coordinates))[0]])
            matrix = cal_distance(coor_part, client)
            distances = distances + matrix[1][0][1:]
            durations = durations + matrix[0][0][1:]
            
        index = distances.index(min(distances))
        rent_address.append(rent['listingSlug'][i])
        school_name.append(school['Stop_name'][index])
        min_distance.append(distances[index])
        min_duration.append(durations[index])
    return rent_address, school_name, min_distance, min_duration

In [109]:
get_coordinate(rent, school)[0]

[[146.666601, -38.617713], [142.590393, -38.386281], [143.475649, -37.084502], [145.234722, -36.901368], [145.21398, -37.742675], [144.342099, -38.170672], [144.4431207, -37.67302961], [143.853936, -37.56401278], [146.88982, -36.129214], [145.354417, -38.044094]]


ApiError: 400 ({'error': {'code': 6003, 'message': "Parameter 'locations' has incorrect value or format."}, 'info': {'engine': {'version': '6.7.0', 'build_date': '2022-02-18T19:37:41Z'}, 'timestamp': 1663313511548}})

In [93]:
data = {'rent_address': get_coordinate(rent, school)[0], 'stop_name': get_coordinate(rent, school)[1], 'min_distance': get_coordinate(rent, school)[2], 'min_duration': get_coordinate(rent, school)[3]}
df = pd.DataFrame(data)

df.to_csv("../data/curated/distances_durations_school21.csv", index=False)

ApiError: 400 ({'error': {'code': 6003, 'message': "Parameter 'locations' has incorrect value or format."}, 'info': {'engine': {'version': '6.7.0', 'build_date': '2022-02-18T19:37:41Z'}, 'timestamp': 1663312834439}})

In [46]:
train = pd.read_excel(f'{raw_dir}train_station_19to20.xlsx')
train.head()

,Fin_year,Stop_ID,Stop_name,Stop_lat,Stop_long,Pax_annual
0,FY19-20,19980,Melton Railway Station (Melton South),-37.703362,144.572217,653450
1,FY19-20,19981,Rockbank Railway Station (Rockbank),-37.729258,144.650630,57650
2,FY19-20,19982,Deer Park Railway Station (Deer Park),-37.777760,144.772307,395900
3,FY19-20,20020,Ardeer Railway Station (Ardeer),-37.783063,144.802193,46100
4,FY19-20,20287,Albury Railway Station (Albury (NSW)),-36.084263,146.924518,26300


In [40]:
def get_coor(rent, train):
    rent_address = []
    stop_name = []
    min_distance = []
    min_duration = []
    for i in range(10):
        coordinates = []
        distances = []
        durations = []
        for j in range(10):
            coordinates.append([train["Stop_long"][j], train["Stop_lat"][j]])
        for k in range(times(len(train))[1]):
            coor_part = []
            coor_part.append([rent["longitude"][i], rent["latitude"][i]])
            coor_part.extend(coordinates[k*times(len(coordinates))[0]:(k+1)*times(len(coordinates))[0]])
            matrix = cal_distance(coor_part, client)
            time.sleep(4)
            distances = distances + matrix[1][0][1:]
            durations = durations + matrix[0][0][1:]
        index = distances.index(min(distances))
        rent_address.append(rent['listingSlug'][i])
        stop_name.append(train['Stop_name'][index])
        min_distance.append(distances[index])
        min_duration.append(durations[index])
    return rent_address, stop_name, min_distance, min_duration


"""
function: put the required columns into a dataframe
param datas: dataframe
"""
def data_frame(rent, train):
    #get dataframe
    data = {'rent_address': get_coor(rent, train)[0], 'stop_name': get_coor(rent, train)[1], 'min_distance': get_coor(rent, train)[2], 'min_duration': get_coor(rent, train)[3]}
    df = DataFrame(data)
    return df

df = data_frame(rent, train)

ApiError: 400 ({'error': {'code': 6003, 'message': "Parameter 'locations' has incorrect value or format."}, 'info': {'engine': {'version': '6.7.0', 'build_date': '2022-02-18T19:37:41Z'}, 'timestamp': 1663305368433}})